In [ ]:
import pandas as pd
from scipy.io import arff
import pandas as pd
import csv
import numpy as np
# import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize']=[16,10]
from keras.layers import Input, Dense
from keras.models import Model
import tensorflow as tf
from keras.models import model_from_json
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,accuracy_score

In [ ]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = 
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.6)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [ ]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
header_list=[]
for i in range(32):
    header_list.append(str("emb")+str(i))
header_list=['App']+header_list+['Type']

In [ ]:
api_embedding=pd.read_csv('Mal2020graphembeddingfull_94acc.csv',names=header_list)
api_embedding

In [ ]:
permission=pd.read_csv('permission_reduced_mal2020.csv')
permission

In [ ]:
permission = permission.drop('Type' , axis='columns')
permission

In [ ]:
dfinal = permission.merge(api_embedding, how='inner', left_on='App', right_on='App')
dfinal

In [ ]:
# dfinal.to_csv(r'permission_embedding_full.csv',index=False)

In [ ]:
dfinal = dfinal.sample(frac=1)
dfinal

In [ ]:
train_df=dfinal.iloc[:11061]
print(train_df.shape)
test_df=dfinal.iloc[11061:]
print(test_df.shape)

In [ ]:
Label_train = train_df.iloc[:,-1:]
Label_test= test_df.iloc[:,-1:]
Y_train= Label_train['Type'].values.tolist()
Y_test= Label_test['Type'].values.tolist()


In [ ]:
one_hot_targets = np.eye(2)[Y_train]
one_hot_targets_test = np.eye(2)[Y_test]

In [ ]:
print(len(Y_train))
print(len(Y_test))

In [ ]:
X_train_permission=train_df.iloc[:,1:6852]
X_train_embedding=train_df.iloc[:,6852:6884]

In [ ]:
X_test_permission=test_df.iloc[:,1:6852]
X_test_embedding=test_df.iloc[:,6852:6884]

In [ ]:
from __future__ import print_function
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional,SimpleRNN,GRU,Reshape,TimeDistributed,concatenate,BatchNormalization
from keras.layers import Conv1D, GlobalMaxPooling1D, Activation, Flatten, UpSampling1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import imdb
from keras.models import model_from_json
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from keras.optimizers import Adam
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session  
import keras_metrics
import keras
from tensorflow.keras import metrics

In [ ]:
def network(C,emb,lr,beta1,beta2,myepsilon,convs):
    print('Number of layer',C)
    print('emb size',emb)
    print('learning rate',lr)
    print('beta1 size',beta1)
    print('beta2 size',beta2)
    print('epsilon size',myepsilon)
    print('convs size',convs)
    max_features=6851
    maxlen = 6851 # used in google
    batch_size = 32# used in google
    sequence = Input(shape=(max_features,), name='Sequence')
    features = Input(shape=(32,), name='Features')
    bm25_features = Input(shape=(999,), name='BMFeatures')
    #embedding size
    if emb<200:
        emb=100
    elif emb < 300:
        emb=200
    elif emb < 400:
        emb=300
    elif emb <500:
        emb=400
    elif emb <600:
        emb=500
    elif emb <700:
        emb=600
    else:
        emb=700
    #conv size
    if convs<64:
        convs=32
    elif convs<128:
        convs=64
    elif convs<130:
        convs=128
    
    x=Embedding(max_features,emb, input_length=maxlen)(sequence)
    for i in range(0,C):
#         x=Dropout(0.2)(x)
        x=Conv1D(convs,5,padding='same',activation='relu',dilation_rate=1)(x)
        x=MaxPooling1D(pool_size=2)(x)
    output_tensor=GlobalMaxPooling1D()(x)
#     cnn_model = Model(inputs=sequence, outputs=output_tensor)
    merged = concatenate([output_tensor,features])
    final=Dense(96,activation='relu')(merged)
#     final = Dropout(0.5)(final)
    final=Dense(96,activation='relu')(final)
    final=Dense(96,activation='relu')(final)
    final = Dense(1, activation='sigmoid')(final)

    model = Model(inputs=[sequence,features], outputs=[final])
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr,beta_1=beta1,beta_2=beta2,epsilon=myepsilon)

    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=[metrics.Precision(), metrics.Recall(),'accuracy'])

    return model


In [ ]:
model = network(8,300,0.001,0.9,0.999,1e-07,120)

In [ ]:
model.fit([X_train_permission,X_train_embedding],np.array(Y_train) ,
        batch_size=64,
        epochs=1,
        validation_data=([X_test_permission,X_test_embedding],np.array(Y_test)))

In [ ]:
model_json = model.to_json()
with open(r"cnn98-23.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5http://localhost:8891/notebooks/Convolution/Convolution.ipynb#
model.save_weights(r"cnn98-23.h5")
print("Saved model to disk")

In [ ]:
pred=model.predict([X_test_permission,X_test_embedding])

In [ ]:
pred_=[]
for i in pred:
    if i[0]<0.5:
        pred_.append(0)
    else:
        pred_.append(1)

In [ ]:
def get_performance(real_label,pred_label):
    acc=accuracy_score(real_label,pred_label)
    print('Accuracy: %f' % acc)
    print(confusion_matrix(real_label,pred_label))
    precision = precision_score(real_label,pred_label)
    print('Precision: %f' % precision)
    recall = recall_score(real_label,pred_label)
    print('recall: %f' % recall)
    f1score= f1_score(real_label,pred_label)
    print('f1score: %f' % f1score)
    return acc,precision,recall,f1score

In [ ]:
acc,precision,recall,f1score=get_performance(Y_test,pred_)

In [ ]:
# load json and create model
json_file = open(r"cnn98-23.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
black_box_model = model_from_json(loaded_model_json)
# load weights into new model
black_box_model.load_weights(r"cnn98-23.h5")

In [ ]:
pred=black_box_model.predict([X_test_permission,X_test_embedding])

In [ ]:
pred_=[]
for i in pred:
    if i[0]<0.5:
        pred_.append(0)
    else:
        pred_.append(1)

In [ ]:
def get_performance(real_label,pred_label):
    acc=accuracy_score(real_label,pred_label)
    print('Accuracy: %f' % acc)
    print(confusion_matrix(real_label,pred_label))
    precision = precision_score(real_label,pred_label)
    print('Precision: %f' % precision)
    recall = recall_score(real_label,pred_label)
    print('recall: %f' % recall)
    f1score= f1_score(real_label,pred_label)
    print('f1score: %f' % f1score)
    return acc,precision,recall,f1score

In [ ]:
acc,precision,recall,f1score=get_performance(Y_test,pred_)